In [1]:
import numpy as np
import tensorflow as tf
import kerastuner as kt
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline 
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [2]:
df= pd.read_csv("all_data_LSTM_full_feature.csv")
test=pd.read_csv("y_DX_month.csv")
df=df.sort_values(by='RID', ascending=True)
test=test.sort_values(by='RID', ascending=True)
groupby=df.groupby("RID").count()
len(groupby)
df=df.drop(['RID', 'VISCODE2'],axis = 1 )

In [3]:
len(df.columns)

132

In [4]:
num_features=len(df.columns)
y= np.array(test.DX_Month)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(df)
X = scaler.transform(df)

In [5]:
scaler2 = MinMaxScaler(feature_range=(0, 1))
y = scaler2.fit_transform(pd.DataFrame(y))
from sklearn. model_selection import train_test_split
x= X.reshape( 613, 4* num_features)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
     x, y, test_size=0.10, stratify= y, shuffle=True, random_state=None) 

In [7]:

def build_model(hp):
    num_units_min  =  400
    num_units_max  =  1000
    num_units_step =  50

    dropout_min  =  .2
    dropout_max  =  0.5
    dropout_step =  0.1
    
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('unit1',  min_value=num_units_min,
                                                 max_value=num_units_max,
                                                 step=num_units_step),
                                                 activation='relu',
                                                 input_dim=4* num_features, 
                                                 kernel_initializer='normal', 
                                                 kernel_regularizer=keras.regularizers.l2(hp.Choice('reg_rate',values=[0.01, 0.05, 0.1]))
                          
                                               ))
    model.add(layers.Dropout(hp.Float('dropout_1',min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step) ) )
 
 
    model.add(layers.Flatten())
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('unitsdense_'+ str(i),
                                            min_value=200,
                                            max_value=800,
                                            step=200),
                                            activation='relu',
                              kernel_regularizer=keras.regularizers.l2(hp.Choice('reg_rate1',values=[0.01, 0.05, 0.1]))))   
        model.add(layers.Dropout(hp.Float('dropoutdense_'+ str(i), 
                                      min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step)))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[ 1e-4])),
        loss='mean_squared_error',
        metrics=['mean_squared_error', 'mean_absolute_error'])
    return model


In [8]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping

tuner = RandomSearch(
    build_model,
    objective='mean_squared_error',
    max_trials=4,
  
    project_name='hager5')



INFO:tensorflow:Reloading Oracle from existing project .\hager5\oracle.json
INFO:tensorflow:Reloading Tuner from .\hager5\tuner0.json


In [9]:

early_stopping = EarlyStopping(monitor='mean_squared_error', patience=30, verbose=1)
callback_list = [ early_stopping ]
tuner.search(X_train, y_train,
             epochs=60,
             batch_size=50, 
             callbacks=callback_list,
             validation_data=(X_test,y_test))
model = tuner.get_best_models(num_models=1)[0]


INFO:tensorflow:Oracle triggered exit

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002BFDFC76FD0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002BFE1FD0970>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002BFE1F88340> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002BFE0F95B80>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002BFE1F888E0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002BFE0F20430>).


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 650)               343850    
_________________________________________________________________
dropout (Dropout)            (None, 650)               0         
_________________________________________________________________
flatten (Flatten)            (None, 650)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               260400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0

In [12]:
print("Best Model")
tuner.get_best_hyperparameters()[0].values

{'unit1': 650,
 'reg_rate': 0.01,
 'dropout_1': 0.2,
 'num_layers': 2,
 'unitsdense_0': 400,
 'reg_rate1': 0.05,
 'dropoutdense_0': 0.4000000000000001,
 'learning_rate': 0.0001,
 'unitsdense_1': 400,
 'dropoutdense_1': 0.5000000000000001,
 'unitsdense_2': 200,
 'dropoutdense_2': 0.30000000000000004,
 'unitsdense_3': 200,
 'dropoutdense_3': 0.30000000000000004}

In [13]:
best_model = tuner.get_best_models()[0]


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002BFE0EF2280> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002BFE1F88F40>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002BFE1FD05E0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002BFE044BF70>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002BFE2043AF0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002BFE2049400>).
2/2 [==============================] - 1s 6ms/step - loss: 5.5614 - mean_squared_error: 0.0473 - mean_absolute_error: 0.1366


0.05947320908308029

In [27]:
my_dict_Train={"RMSE": [],"MSE": [],"MBE": []};
my_dict_Test={"RMSE": [],"MSE": [],"MBE": []};

In [33]:
for i in range(0,2):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.10, stratify= y, shuffle=True, random_state=None)
        history= best_model.fit(X_train, y_train, epochs=15, batch_size=50,  verbose=1,shuffle=True, validation_data=(X_test,y_test))
        my_dict_Train["RMSE"].append(np.sqrt(np.mean(history.history['val_mean_squared_error'])) )
        my_dict_Train["MSE"].append(np.mean(history.history['val_mean_squared_error']))
        my_dict_Train["MBE"].append(np.mean(history.history['val_mean_absolute_error']))
        y_pred = best_model.predict(X_test)
        my_dict_Test["RMSE"].append(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
        my_dict_Test["MSE"].append(metrics.mean_squared_error(y_test,y_pred))
        my_dict_Test["MBE"].append(metrics.mean_absolute_error(y_test,y_pred))
        best_model.reset_states()

Epoch 1/15
12/12 [==============================] - 0s 26ms/step - loss: 0.0663 - mean_squared_error: 0.0649 - mean_absolute_error: 0.1743 - val_loss: 0.0655 - val_mean_squared_error: 0.0641 - val_mean_absolute_error: 0.1721
Epoch 2/15
12/12 [==============================] - 0s 17ms/step - loss: 0.0664 - mean_squared_error: 0.0650 - mean_absolute_error: 0.1736 - val_loss: 0.0654 - val_mean_squared_error: 0.0641 - val_mean_absolute_error: 0.1714
Epoch 3/15
12/12 [==============================] - 0s 18ms/step - loss: 0.0664 - mean_squared_error: 0.0651 - mean_absolute_error: 0.1733 - val_loss: 0.0653 - val_mean_squared_error: 0.0641 - val_mean_absolute_error: 0.1705
Epoch 4/15
12/12 [==============================] - 0s 19ms/step - loss: 0.0664 - mean_squared_error: 0.0652 - mean_absolute_error: 0.1729 - val_loss: 0.0653 - val_mean_squared_error: 0.0641 - val_mean_absolute_error: 0.1702
Epoch 5/15
12/12 [==============================] - 0s 18ms/step - loss: 0.0660 - mean_squared_error

In [34]:
print("Training result")
for metric in my_dict_Train.keys():
    print("%s: %.3f" % (metric, np.mean(my_dict_Train[metric])))
   

Training result
RMSE: 0.253
MSE: 0.064
MBE: 0.170


In [35]:
print("Testing result")
for metric in my_dict_Test.keys():
    print("%s: %.3f" % (metric, np.mean(my_dict_Test[metric])))

Testing result
RMSE: 0.253
MSE: 0.064
MBE: 0.169
